# Experiment

In [ ]:
#| default_exp ml.experiment

In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#| export
def context_wandb(entity, project, **kwargs):
    import wandb
    try:
        wandb_run = wandb.init(
            entity=entity,
            project=project,
            **kwargs
        )
        yield wandb_run
    finally:
        if wandb_run:
            wandb.finish()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()